In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

!cp -r gdrive/MyDrive/clean-dirty-garbage-containers-V6.1 .

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
! pip install torch-lr-finder

In [ ]:
from data import get_dataloaders, MontevideoDirtyContainerDataset
from models import load_model, perform_training

In [ ]:
model = load_model("resnet18", [1024,256,32], 0.4, freeze=True)

In [ ]:
import torch

train, val, test = get_dataloaders(16,"clean-dirty-garbage-containers-V6.1","clean-dirty-garbage-containers-V6.1/clean-dirty-garbage-containers/clean-dirty-metadata.csv", use_augmentation=False)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model.to(device)
out1 = perform_training(model, train, val, criterion, optimizer, 10, device, use_scheduler=True, scheduler_patience=2, auto_lr_find=False)

In [ ]:
from models import unfreeze_model

unfreeze_model(model)

In [ ]:
train, val, test = get_dataloaders(16,"clean-dirty-garbage-containers-V6.1","clean-dirty-garbage-containers-V6.1/clean-dirty-garbage-containers/clean-dirty-metadata.csv", use_augmentation=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
out2 = perform_training(model, train, val, criterion, optimizer, 20, device, use_scheduler=True, scheduler_patience=2, auto_lr_find=False)

In [ ]:
train_losses = out1[0] + out2[0]
val_losses = out1[1] + out2[1]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses)
plt.plot(val_losses)
# plt.savefig("gdrive/MyDrive/results/[XYXYXY].jpg")

In [ ]:
from models import perform_testing

preds, labels = perform_testing(model.to(device), test, device)

In [ ]:
from evaluation import evaluate_predictions, plot_false_predictions

evaluate_predictions(preds, labels)

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
    ])
test_set = MontevideoDirtyContainerDataset("clean-dirty-garbage-containers-V6.1/clean-dirty-garbage-containers/clean-dirty-metadata.csv","clean-dirty-garbage-containers-V6.1",transform,"test")

plot_false_predictions(preds, labels, test_set, device)

## Detecting Garbage using Grad-CAMs

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import cv2
import torch.nn.functional as F

In [ ]:
from interpretation import plot_grad_cam

plot_grad_cam(5, 5, model, test, device)

In [ ]:
from models import save_model

save_model(model, "resnet18-0.925-1024-256-32", "/content/gdrive/MyDrive/models/")

## Evaluation and Plots

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

!cp -r gdrive/MyDrive/clean-dirty-garbage-containers-V6.1 .

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install torch-lr-finder

In [ ]:
from data import get_dataloaders, MontevideoDirtyContainerDataset

train, val, test = get_dataloaders(16,"clean-dirty-garbage-containers-V6.1","clean-dirty-garbage-containers-V6.1/clean-dirty-garbage-containers/clean-dirty-metadata.csv", use_augmentation=False)

In [ ]:
from models import load_model_from_saved

model = load_model_from_saved("efficientnet-b0", [1024,256,32], 0.4, "/content/gdrive/MyDrive/models/", "efficientb0-0.935-1024-256-32", device)

In [ ]:
from models import perform_testing

preds, labels = perform_testing(model.to(device), test, device)

In [ ]:
from evaluation import evaluate_predictions, plot_false_predictions

res = evaluate_predictions(preds, labels)

In [ ]:
from torchvision import transforms
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
    ])
test_set = MontevideoDirtyContainerDataset("clean-dirty-garbage-containers-V6.1/clean-dirty-garbage-containers/clean-dirty-metadata.csv","clean-dirty-garbage-containers-V6.1",transform,"test")

fig = plot_false_predictions(preds, labels, test_set, device)
# fig.savefig("/content/gdrive/MyDrive/results/[XYXYXY].jpg", dpi=400)

In [ ]:
from interpretation import plot_predictions

fig = plot_predictions(preds, labels, test_set, 6, 5, model, device)
# fig.savefig("/content/gdrive/MyDrive/results/[XYXYXY].jpg", dpi=400)

## Cleaning the streets using Image Inpainting

In [ ]:
it = iter(test)
dirty_list = []
for i in range(40):
  img, label, _  = next(it)
  for j in range(len(label)):
    if label[j] == 1:
      dirty_list.append(img[j])

In [ ]:
from google.colab.patches import cv2_imshow
from interpretation import ResNet_CAM
import numpy as np
import cv2
import torchvision
from torchvision import transforms

inv_norm = transforms.Compose([transforms.Normalize(mean=[0., 0., 0.],
                                                        std=[1 / 0.229, 1 / 0.224, 1 / 0.225]),
                                   transforms.Normalize(mean=[-0.485, -0.456, -0.406],
                                                        std=[1., 1., 1.]), ])
def get_inpaint_images(net, img, threshold):
  net.eval()
  pred = net(img)
  pred[:, pred.argmax(dim=1)].backward()
  gradients = net.get_activations_gradient()
  pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
  activations = net.get_activations(img).detach()
  for i in range(activations.size(1)):
      activations[:, i, :, :] *= pooled_gradients[i]
  heatmap = torch.mean(activations, dim=1).squeeze()
  heatmap = np.maximum(heatmap, 0)
  heatmap /= torch.max(heatmap)
  
  heatmap = cv2.resize(heatmap.numpy(), (img.shape[2], img.shape[3]))
  heatmap = torch.tensor(heatmap)

  img1 = torch.where(heatmap>threshold, 0, img)
  img_mask = torch.where(heatmap>threshold, 255.0, torch.zeros(img.size()))
  torchvision.utils.save_image(inv_norm(img1), "img_to_inpaint" + str(layer_k) + ".jpg")
  torchvision.utils.save_image(img_mask, "mask" + str(layer_k) + ".jpg")

layer_k = 6
threshold = 0.2

model.train()
baseline_cam_net = ResNet_CAM(model, layer_k)
baseline_cam_net.to("cpu")

i = 10
img = dirty_list[i][None]
get_inpaint_images(baseline_cam_net, img, threshold)

img_to_inpaint = cv2.imread(f"./img_to_inpaint{layer_k}.jpg")
mask = cv2.imread(f'./mask{layer_k}.jpg',0)

dst = cv2.inpaint(img_to_inpaint,mask,3,cv2.INPAINT_NS)

cv2_imshow((inv_norm(img)*256)[0].permute(1,2,0).numpy())
cv2_imshow(dst)
# cv2.imwrite(f"/content/gdrive/MyDrive/results/ex{i}-dirty.jpg", (inv_norm(img)*256)[0].permute(1,2,0).numpy())
# cv2.imwrite(f"/content/gdrive/MyDrive/results/ex{i}-clean.jpg", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()